In [10]:
import heapq
import matplotlib
from matplotlib import patches
from matplotlib.patches import Rectangle
from matplotlib import animation
from matplotlib.animation import FuncAnimation
import tkinter as tk
from tkinter import ttk
from matplotlib.widgets import Button
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import random
import numpy as np
import itertools
import numpy as np
import gym
from gym import spaces
import time

class VehicleAgent:
    def __init__(self, unique_id, pos, direction, speed, color, grid, goal, active=True):
        self.unique_id = unique_id
        self.pos = pos
        self.direction = direction
        self.goal = goal
        self.speed = speed
        self.color = color
        self.path_index = 0
        self.waiting = False
        self.priority = None
        self.yield_time = None
        self.grid = grid
        self.messages_received = []
        self.goal = goal
        self.priority = 0.0
        self.crossroad_passed = False
        self.arrival_time = None
        self.creation_time = time.time()

    def has_reached_goal(self):
        return self.pos == self.goal

    def calculate_distance_to_goal(self):
        return abs(self.pos[0] - self.goal[0]) + abs(self.pos[1] - self.goal[1])

    def calculate_priority(self):
        distance_score = 100 / (1 + self.calculate_distance_to_goal())
        speed_score = self.speed
        random_score = random.uniform(0, 0.00001)
        creation_time_score = self.creation_time
        self.priority = distance_score + speed_score + random_score + creation_time_score
        if self.arrival_time is not None:
            self.priority += 100

    def predict_future_positions(self, steps=5):
        future_positions = []
        future_positions.append(self.pos)
        for step in range(1, steps + 5):
            future_pos = None
            if self.direction in ['N', 'north_to_south']:
                future_pos = (self.pos[0], self.pos[1] + step)
            elif self.direction in ['S', 'south_to_north']:
                future_pos = (self.pos[0], self.pos[1] - step)
            elif self.direction in ['E', 'east_to_west']:
                future_pos = (self.pos[0] - step, self.pos[1])
            elif self.direction in ['W', 'west_to_east']:
                future_pos = (self.pos[0] + step, self.pos[1])
            else:
                raise ValueError("Invalid direction: {}".format(self.direction))
            if future_pos:
                future_positions.append(future_pos)
                if len(future_positions) >= steps + 1:
                    break
        return future_positions

    def calculate_future_position(self, steps):
        future_pos = None
        if self.direction in ['N', 'north_to_south']:
            future_pos = (self.pos[0], self.pos[1] + steps)
        elif self.direction in ['S', 'south_to_north']:
            future_pos = (self.pos[0], self.pos[1] - steps)
        elif self.direction in ['E', 'east_to_west']:
            future_pos = (self.pos[0] - steps, self.pos[1])
        elif self.direction in ['W', 'west_to_east']:
            future_pos = (self.pos[0] + steps, self.pos[1])
        return future_pos

    def send_message(self, message, vehicles):
        for vehicle in vehicles:
            if vehicle.unique_id != self.unique_id:
                vehicle.receive_message(message)

    def check_future_conflict(self, other_vehicle, safe_distance=1.5):
        if not (self.grid.is_in_crossroad(self.pos) or self.grid.is_in_crossroad(other_vehicle.pos)):
            return False
        current_distance = distance_between(self.pos, other_vehicle.pos)
        if current_distance <= safe_distance:
            return True
        for step in range(1, 4):
            my_future_pos = self.calculate_future_position(step)
            other_future_pos = other_vehicle.calculate_future_position(step)
            distance1 = distance_between(my_future_pos, other_future_pos)
            distance2 = distance_between(my_future_pos, other_vehicle.pos)
            if distance1 <= safe_distance or distance2 <= safe_distance:
                return True
        return False

    def check_collision_and_decide(self):
        for message in self.messages_received:
            other_vehicle_future_positions = message['future_positions']
            for future_position in self.predict_future_positions():
                if future_position in other_vehicle_future_positions:
                    self.waiting = True
                    return
        self.waiting = False

    def broadcast(self, vehicles):
        self.calculate_priority()
        future_positions = self.predict_future_positions()
        message = {
            'sender': self.unique_id,
            'current_position': self.pos,
            'future_positions': future_positions,
            'priority': self.priority,
            'waiting': self.waiting
        }
        print(f"Vehicle {self.unique_id} broadcasting message: {message}")
        self.send_message(message, vehicles)

    def broadcast_stop_intent(self, vehicles):
        message = {
            'type': 'stop_intent',
            'sender': self.unique_id,
            'position': self.pos,
        }
        for vehicle in vehicles:
            if vehicle.unique_id != self.unique_id:
                vehicle.receive_message(message)

    def receive_message(self, message):
        self.messages_received.append(message)

    def move_towards_goal(self):
        dx, dy = 0, 0
        if self.pos[0] < self.goal[0]:
            dx = 1
        elif self.pos[0] > self.goal[0]:
            dx = -1
        if self.pos[1] < self.goal[1]:
            dy = 1
        elif self.pos[1] > self.goal[1]:
            dy = -1
        self.pos = (self.pos[0] + dx, self.pos[1] + dy)

    def execute_action(self, action):
        if action == 0 and self.pos[1] > 0:
            self.pos = (self.pos[0], self.pos[1] - 1)
        elif action == 1 and self.pos[1] < grid.height - 1:
            self.pos = (self.pos[0], self.pos[1] + 1)
        elif action == 2 and self.pos[0] > 0:
            self.pos = (self.pos[0] - 1, self.pos[1])
        elif action == 3 and self.pos[0] < grid.width - 1:
            self.pos = (self.pos[0] + 1, self.pos[1])

    def move(self, vehicles):
        self.calculate_priority()
        self.broadcast(vehicles)
        conflict = False
        for vehicle in vehicles:
            if vehicle.unique_id == self.unique_id:
                continue
            for step in range(1, 4):
                future_pos = self.calculate_future_position(step)
                if future_pos == vehicle.pos and self.grid.is_in_crossroad(future_pos):
                    conflict = True
                    break
            if conflict:
                break
        if conflict:
            self.waiting = True
        else:
            self.waiting = False
            self.move_towards_goal()
        if self.waiting:
            self.broadcast_stop_intent(vehicles)

def create_vehicle_for_specific_route(grid, entry_exit_points):
    direction = random.choice(list(entry_exit_points.keys()))
    route_info = entry_exit_points[direction]
    start = route_info['start'][0]
    end = route_info['end'][0]
    speed = random.randint(1, 2)
    color = random.choice(["blue"])
    vehicle_id = f"vehicle_{direction}"
    return VehicleAgent(vehicle_id, start, direction, speed, color, grid, end)

grid = Gridworld(20, 20)
road_width = 4
entry_exit_points = setup_roadway_entry_exit_points(grid, 9)
num_vehicles = random.randint(1, 2)
vehicles = [create_vehicle_for_specific_route(grid, entry_exit_points) for _ in range(num_vehicles)]
fig, ax = plt.subplots()

new_vehicle_interval = 5
time_since_last_new_vehicle = 0

traffic =np.arange(0，41，1)vehicle_type =np.arange(0，2，1)priority increase =np.arange(0，101，1)
# Define membership functionstraffic low=fuzz.trimf(traffic，[0，0，8])traffic high =fuzz.trimf(traffic,[8，40，40])
vehicle_type_normal=fuzz.trimf(vehicle type,[0，0，0.5])vehicle type emergency =fuzz.trimf(vehicle type,[0.5,1，1])
priority_increase_none = fuzz.trimf(priority increase,[0,0，30])priority_increase_medium = fuzz.trimf(priority_increase,[20,50,80])priority increase high = fuzz.trimf(priority increase,[70,100,100])
# Define fuzzy rulesrule1 = ctrl.Rule(traffic['high']& vehicle type['emergency'], priorityrule2 = ctrl.Rule(traffic['high'], priority increase['medium'])
#Create control system and simulationpriority control=ctrl.ControlSystem([rule1,rule2])priority simulator = ctrl.ControlSystemSimulation(priority control)

def handle_vehicle_actions():
    global vehicles, time_since_last_new_vehicle
    for vehicle in vehicles:
        vehicle.messages_received = []
    for vehicle in vehicles:
        vehicle.broadcast(vehicles)
    for vehicle in vehicles:
        vehicle.check_collision_and_decide()
    for vehicle in vehicles:
        vehicle.move(vehicles)
    vehicles = [vehicle for vehicle in vehicles if not vehicle.has_reached_goal()]

def update(frame):
    global current_time
    current_time += 1
    global time_since_last_new_vehicle
    ax.clear()
    ax.set_xlim(0, grid.width)
    ax.set_ylim(0, grid.height)
    setup_crossroad_obstacles(grid, 9, 4)
    draw_grid_with_crossroad(grid, ax)
    handle_vehicle_actions()
    time_since_last_new_vehicle += 1
    if time_since_last_new_vehicle >= new_vehicle_interval:
        try_to_add_new_vehicles()
        time_since_last_new_vehicle = 0
    for vehicle in vehicles:
        ax.add_patch(Rectangle((vehicle.pos[0]-0.5, vehicle.pos[1]-0.5), 1, 1, color=vehicle.color))
    ax.set_title("Vehicle Movement Simulation")
    plt.gca().invert_yaxis()

def try_to_add_new_vehicles():
    global vehicles
    num_new_vehicles = random.randint(1, 5)
    for _ in range(num_new_vehicles):
        direction, info = random.choice(list(entry_exit_points.items()))
        start = info['start'][0]
        if any(vehicle.pos == start for vehicle in vehicles):
            continue
        end = info['end'][0]
        speed = 1
        color = random.choice(["blue"])
        vehicle_id = f"vehicle_{len(vehicles) + 1}"
        new_vehicle = VehicleAgent(vehicle_id, start, direction, speed, color, grid, end)
        vehicles.append(new_vehicle)

def add_high_priority_vehicle(event):
    global vehicles
    direction, info = random.choice(list(entry_exit_points.items()))
    start = info['start'][0]
    end = info['end'][0]
    if not any(vehicle.pos == start for vehicle in vehicles):
        new_vehicle = VehicleAgent(f"high_priority_vehicle_{len(vehicles) + 1}", start, direction, 2, "red", grid, end)
        new_vehicle.priority = float('inf')
        vehicles.append(new_vehicle)

ani = FuncAnimation(fig, update, frames=range(1000), repeat=False)

button_ax = plt.axes([0.8, 0.025, 0.1, 0.04])
button = Button(button_ax, 'Add Red Car')
button.on_clicked(add_high_priority_vehicle)

plt.show()


SyntaxError: invalid character in identifier (154099746.py, line 207)